In [ ]:
import json
import pandas as pd
from os.path import join
from scipy.io import savemat
import numpy as np
import itertools
from surgeon_recording.reader import Reader
from scipy.io import savemat
from sklearn.preprocessing import MinMaxScaler

In [ ]:
reader = Reader()

In [ ]:
datafolder = join('..', 'data', 'cuts')

In [ ]:
nb_files = 20
exp_folders = [join(datafolder, 'cut' + str(i)) for i in (np.arange(nb_files) + 2)]

In [ ]:
exp_folders

In [ ]:
def insert_row(data, row, labels=None):
    return data.append(pd.Series(row, labels), ignore_index=True)

In [ ]:
def preprocess_motion(data, frames_list, scaler):
    column_list = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z') for f in frames_list))
    data[column_list] = 2 * scaler.fit_transform(data[column_list]) - 1

In [ ]:
def downsample(data, time_vector):
    current_time_index = 0
    downsampled_data = pd.DataFrame(columns=data.columns)
    
    for i in range(len(time_vector)):
        t = time_vector.iloc[i]
        start_time = current_time_index
        
        while current_time_index < data.shape[0] and data['relative_time'].iloc[current_time_index] < t:
            current_time_index = current_time_index + 1
        stop_time = current_time_index

        average_data = data.iloc[start_time:stop_time, :].mean()
        downsampled_data = insert_row(downsampled_data, average_data)
    return downsampled_data

In [ ]:
def preprocess_emg(data, emg_list, scaler):
    # first take the absolute value of the data
    data[emg_list] = data[emg_list].abs()
    # then apply a minmax scaler
    data[emg_list] = 2 * scaler.fit_transform(data[emg_list]) - 1

In [ ]:
frames_list = ['blade', 'wrist']
optitrack_list = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z',
                                                     f + '_qx', f + '_qy', f + '_qz', f + '_qw')
                                                    for f in frames_list))
emg_list = ['emg' + str(i) for i in range(8)]

In [ ]:
timeseries = []

scalers = {}
scalers['optitrack'] = MinMaxScaler()
scalers['emg'] = MinMaxScaler()

for folder in exp_folders:
    reader.play(folder)
    # extract optitrack data and preprocess them
    opt_data = reader.data['optitrack'].reset_index(drop=True)
    preprocess_motion(opt_data, frames_list, scalers['optitrack'])
    # downsample emg and preprocess it
    emg_data = downsample(reader.data['emg'], opt_data['relative_time'])
    preprocess_emg(emg_data, emg_list, scalers['emg'])
    # merge the data
    merge_data = pd.concat([opt_data, emg_data.iloc[:, 3:]], axis=1)
    # store in the list
    timeseries.append(merge_data)

# Save as mat file for Nadia's segmentation

In [ ]:
merged_column_list = optitrack_list

# drop the data in matlab
matdata = {}
matdata['timeseries'] = []
for t in timeseries:
    # drop the index and times columns and convert to numpy array
    matdata['timeseries'].append(t[merged_column_list].to_numpy())

In [ ]:
savemat("segmentation_data.mat", matdata)

# Apply 0 velocity segmentation

## Compute velocities

In [ ]:
for i, data in enumerate(timeseries):
    frame_list = ['blade', 'wrist']

    header = list(itertools.chain.from_iterable((f + '_dx', f + '_dy', f + '_dz', f + '_vel_mag',
                                                 f + '_ddx', f + '_ddy', f + '_ddz', f + '_acc_mag')
                                                for f in frame_list))

    derived_data = pd.DataFrame(columns=header)

    for index, row in data.iterrows():
        row_data = []
        for f in frame_list:
            # compute velocity
            if index == 0:
                vel = np.zeros(3)
            else:
                x0 = data.loc[index - 1, [f + '_x', f + '_y', f + '_z']].to_numpy()
                x1 = row.loc[[f + '_x', f + '_y', f + '_z']].to_numpy()
                dt = row['relative_time'] - data.loc[index - 1, 'relative_time']
                vel = (x1 - x0) / dt
            vel_mag = np.linalg.norm(vel)
            # compute accleration
            if index < 2:
                acc = np.zeros(3)
            else:
                x0 = data.loc[index - 2, [f + '_x', f + '_y', f + '_z']].to_numpy()
                x1 = data.loc[index - 1, [f + '_x', f + '_y', f + '_z']].to_numpy()
                x2 = row.loc[[f + '_x', f + '_y', f + '_z']].to_numpy()
                dt = (row['relative_time'] - data.loc[index - 2, 'relative_time']) / 2.
                acc = (x2 - 2*x1 + x0) / (dt *  dt)
            acc_mag = np.linalg.norm(acc)
            frame_vector = np.hstack((vel, vel_mag, acc, acc_mag))
            row_data = np.hstack((row_data, frame_vector))
        derived_data = insert_row(derived_data, row_data, header)
        timeseries[i] = pd.concat([data, derived_data], axis=1)

## compute normed and rolling average

In [ ]:
vel_frames = list(itertools.chain.from_iterable((f + '_vel_mag', f + '_acc_mag') for f in frame_list))

In [ ]:
window_size = 10

In [ ]:
normed_velocities = []
rolling_average_velocities = []

for data in timeseries:
    vel_acc_data = data.loc[:, vel_frames]
    normed_velocities.append((vel_acc_data - vel_acc_data.min()) / (vel_acc_data.max() - vel_acc_data.min()))
    rolling_average_velocities.append(normed_velocities[-1].rolling(window=window_size).mean().iloc[window_size-1:])

In [ ]:
idx = 1
normed_velocities[idx].plot.line()
rolling_average_velocities[idx].plot.line()